<a href="https://colab.research.google.com/github/KonradMurach/dw_matrix/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 3.5MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from google.colab import drive

from ast import literal_eval
from tqdm import tqdm_notebook

In [0]:
#drive.mount("/content/drive")

In [9]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [10]:
ls

data/  HelloGithub.ipynb  LICENSE  matrix_one/  README.md  w3day3.ipynb


In [11]:
ls data

shoes_prices.csv


In [0]:
df = pd.read_csv('data/shoes_prices.csv', low_memory=False)

In [0]:
def run_model(feats, model=DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [27]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [28]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [30]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [0]:
def parse_teatures(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict

  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict

df['features_parsed'] = df['features'].map(parse_teatures) 

In [44]:
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [45]:
keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()) )
len(keys)

476

In [48]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [49]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_sole', 'feat_retail price', 'feat_chest size', 'feat_main color',
       'feat_fuel type', 'feat_black', 'feat_amazonbestsellersrank',
       'feat_place of origin', 'feat_hairstyle',
       'feat_high visibility (ansi compliant)'],
      dtype='object', length=526)

In [54]:
df[False == df['feat_athlete'].isnull()].shape[0] / df.shape[0] * 100

0.0437636761487965

In [51]:
df.shape[0]

18280

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

In [61]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [91]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key)+'_cat' = df[get_name_feat(key].factorize()[0]


SyntaxError: ignored

In [66]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
df[df.brand == df.feat_brand][['brand', 'feat_brand']].head()

,brand,feat_brand
0,josmo,josmo
1,josmo,josmo
2,servus by honeywell,servus by honeywell
3,servus by honeywell,servus by honeywell
4,servus by honeywell,servus by honeywell


In [67]:
df[df.brand == df.feat_brand].shape

(8846, 531)

In [0]:
feats_cat = [x for x in df.columns if 'cat' in x]

In [99]:
feats_cat

['categories',
 'brand_cat',
 'feat_shoe category',
 'feat_multi pack indicator',
 'feat_catalog',
 'feat_location - country',
 'feat_certifications and listings',
 'feat_location - city/state',
 'feat_fabrication',
 'feat_clothing category',
 'feat_recommended location',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat']

In [107]:
feats = ['brand_cat','feat_brand_cat','feat_gender_cat','feat_material_cat', 'feat_style_cat']
#feats += feats_cat
#feats = list(set(feats))

model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
result = run_model(feats, model)
result

(-57.174254322129265, 4.237153476213779)

In [108]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X, y)
eli5.show_weights(perm, feature_names=feats)

(-57.174254322129265, 4.237153476213779)


Weight,Feature
0.2605 ± 0.0099,brand_cat
0.1070 ± 0.0093,feat_material_cat
0.0452 ± 0.0026,feat_gender_cat
0.0207 ± 0.0011,feat_brand_cat
0.0064 ± 0.0007,feat_style_cat


In [81]:
df['brand'].value_counts(normalize=True)

nike                  0.097210
puma                  0.033315
ralph lauren          0.028775
vans                  0.021116
new balance           0.020295
                        ...   
jockey                0.000055
adidas performance    0.000055
banuce                0.000055
p.ku.vdsl             0.000055
unbrand               0.000055
Name: brand, Length: 1732, dtype: float64

In [90]:
df[df['brand'] == 'nike'].features_parsed.sample(10).values

array([{'shoe size': '8 d(m) us', 'gender': 'men', 'size': '8 d(m) us', 'shoe category': 'men', 'brand': 'nike', 'color': 'game royal/ white-university red', 'model': '884726322819'},
       {}, {},
       {'manufacturer part number': '749809400', 'gender': 'boys', 'shoe category': 'boys shoes', 'brand': 'nike', 'color': 'mid navy/dark obsidian/total orange', 'casual & dress shoe style': 'athletic', 'shoe width': 'medium'},
       {},
       {'gender': 'men', 'color': 'black', 'condition': 'new with tags'},
       {'sport': 'soccer', 'condition': 'new with box'}, {},
       {'sport': 'training shoes', 'occasion': 'training shoes', 'material': 'synthetic-and-mesh', 'gender': 'men', 'shoe size': '9.5 us men', 'shoe category': "men's shoes", 'assembled product dimensions (l x w x h)': '12.00 x 8.00 x 6.00 inches', 'fabric content': 'synthetic-and-mesh', 'color': 'poison gree/hyper blue/white', 'model': '553684 302', 'shoe closure': 'lace-up', 'casual & dress shoe style': 'cross trainer sh

In [114]:
ls matrix_one

day3.ipynb  day5.ipynb


In [0]:
!git add matrix_one/day5.ipynb

In [121]:
!git commit -m "add day5"

[master 9b44ba2] add day5
 1 file changed, 1 insertion(+)
 create mode 100644 matrix_one/day5.ipynb


fatal: remote origin already exists.
